In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from datetime import datetime,timedelta

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [19]:
querys={
    
    'TabMargem':
    
    """
    
    SELECT * FROM netfeira.vw_targetmargem
    
    """,
    
    'SELLIN':
    
    """
    
    SELECT * FROM netfeira.vw_sellin
    WHERE Empresa='NETFEIRA'

    """,
    
    'Metas':
    
    """
    
    SELECT * FROM netfeira.vw_metas
    
    """,
    
    'Calendario':
    
    """
    
    SELECT * FROM netfeira.vw_calend
    WHERE Ano=YEAR(GETDATE()) AND [ID Mês]=MONTH(GETDATE())
    
    """
    
    
}

In [20]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [21]:
def Classificacao(val):
    
    if(val<=80):
        
        tipo='A'
        
        pass
    
    elif(val<=95):
        
        tipo='B'
        
        pass
    
    else:
        
        tipo='C'
        
        pass
    
    return tipo
    
    
    pass

In [22]:
perc_meta=1-round(tabelas_df['TabMargem']['Margem Média'].mean(),4)

perc_meta

0.6656

In [23]:
total=tabelas_df['Metas']['Meta R$'].sum()

val_meta=round(total*perc_meta,2)

val_meta

0.0

In [24]:
tabelas_df['Calendario'].columns

Index(['Data', 'Dia Útil', 'Data Trabalhada', 'Ano', 'ID Mês', 'Mês',
       'Mês Resumido', 'Dia', 'Mês Meta', 'ID Semana', 'Semana',
       'Trimestre Ano', 'Semestre Ano', 'Semana Ano'],
      dtype='object')

In [25]:
semana_atu=tabelas_df['Calendario']['Semana Ano'].loc[(tabelas_df['Calendario']['Ano']==datetime.now().year)&(tabelas_df['Calendario']['ID Mês']==datetime.now().month)&(tabelas_df['Calendario']['Dia']==datetime.now().day)].max()

list_datas=tabelas_df['Calendario']['Data'].loc[tabelas_df['Calendario']['Semana Ano']==semana_atu].loc[tabelas_df['Calendario']['Dia Útil']==True].tolist()

list_datas

[Timestamp('2022-11-01 00:00:00'),
 Timestamp('2022-11-03 00:00:00'),
 Timestamp('2022-11-04 00:00:00')]

In [30]:
tabelas_df['Semana Ano']=tabelas_df['Calendario'][['Semana Ano','Dia','Data Trabalhada']].loc[tabelas_df['Calendario']['Dia Útil']==True].groupby(['Semana Ano'],as_index=False).count()

tabelas_df['Semana Ano']['Positivado']=tabelas_df['Semana Ano'].apply(lambda info: 1 if info['Dia']==info['Data Trabalhada'] else 0,axis=1)

semana_util=tabelas_df['Semana Ano']['Semana Ano'].count()

semana_trab=tabelas_df['Semana Ano']['Semana Ano'].loc[tabelas_df['Semana Ano']['Positivado']==1].count()

dia_util=tabelas_df['Semana Ano']['Dia'].sum()

valores=[]

for i in tabelas_df['Semana Ano']['Semana Ano'].tolist():
    
    list_datas=tabelas_df['Calendario']['Data'].loc[tabelas_df['Calendario']['Semana Ano']==i].loc[tabelas_df['Calendario']['Dia Útil']==True].tolist()
    
    val_semana=round(tabelas_df['SELLIN']['Total NFe'].loc[(tabelas_df['SELLIN']['Tipo']=='PADRAO')&(tabelas_df['SELLIN']['Data de Recebimento'].isin(list_datas))].sum(),2)
    
    valores.append(val_semana)
    
    pass

tabelas_df['Semana Ano']['Meta Semanal']=round((val_meta/dia_util)*tabelas_df['Semana Ano']['Dia'],2)

tabelas_df['Semana Ano']['Valor Semanal']=valores

tabelas_df['Semana Ano']['Dif Semanal']=tabelas_df['Semana Ano']['Meta Semanal']-tabelas_df['Semana Ano']['Valor Semanal']

for i,val in enumerate(tabelas_df['Semana Ano']['Semana Ano'].tolist()):
    
    if(i==0):
        
        valores.append(0)
        
        pass
    
    else:
        
        soma=tabelas_df['Semana Ano']['Dif Semanal'].loc[tabelas_df['Semana Ano']['Semana Ano']==(val-1)].sum()
                
        tabelas_df['Semana Ano'].loc[tabelas_df['Semana Ano']['Semana Ano']==val,'Meta Semanal']=tabelas_df['Semana Ano']['Meta Semanal'].loc[tabelas_df['Semana Ano']['Semana Ano']==val]+soma
        
        pass
    
    pass

tabelas_df['Semana Ano']['Dif Semanal']=tabelas_df['Semana Ano']['Meta Semanal']-tabelas_df['Semana Ano']['Valor Semanal']

tabelas_df['Semana Ano']['Perc %']=round(tabelas_df['Semana Ano']['Valor Semanal']/tabelas_df['Semana Ano']['Meta Semanal'],4)*100

tabelas_df['Semana Ano'].loc[tabelas_df['Semana Ano']['Perc %'].isnull(),'Perc %']=0

tabelas_df['Semana Ano']

,Semana Ano,Dia,Data Trabalhada,Positivado,Meta Semanal,Valor Semanal,Dif Semanal,Perc %
0,45,3,1,0,0.0,0.0,0.0,0.0
1,46,5,0,0,0.0,0.0,0.0,0.0
2,47,4,0,0,0.0,0.0,0.0,0.0
3,48,5,0,0,0.0,0.0,0.0,0.0
4,49,3,0,0,0.0,0.0,0.0,0.0


In [31]:
val_semana=round(tabelas_df['SELLIN']['Total NFe'].loc[(tabelas_df['SELLIN']['Tipo']=='PADRAO')&(tabelas_df['SELLIN']['Data de Recebimento'].isin(list_datas))].sum(),2)

val_semana

0.0

In [32]:
meta_semanal=round((val_meta/dia_util)*len(list_datas),2)

meta_semanal

0.0

In [33]:
tabelas_df['SELLIN'].columns

Index(['Empresa', 'ID Fornecedor', 'CNPJ', 'Fornecedor', 'NFe',
       'Data de Emissão', 'Data de Recebimento', 'Local de Estoque',
       'Tipo de Entrada', 'Tipo', 'SKU', 'Cód. Fabricante', 'Produto',
       'Fabricante', 'Status', 'Seq', 'Unid. CMP', 'Qtde Total',
       'Valor Unitário', 'Total dos Produtos', 'Peso Bruto', 'Peso Líquido',
       'PIS R$', 'COFINS R$', 'IPI R$', 'ICMS R$', 'ICMS ST R$', 'Frete R$',
       'Total NFe'],
      dtype='object')

In [34]:
dt_atual=datetime.now()-timedelta(days=1)

dt_ant=datetime.now()-timedelta(days=7)

In [37]:
tabelas_df['Consolidado']=tabelas_df['SELLIN'].loc[(tabelas_df['SELLIN']['Tipo']=='PADRAO')&(tabelas_df['SELLIN']['Data de Recebimento'].between(dt_ant,dt_atual))]

tabelas_df['Consolidado']=tabelas_df['Consolidado'][['SKU','Produto','Fabricante','Total NFe']].groupby(['SKU','Produto','Fabricante'],as_index=False).sum()

tabelas_df['Consolidado'].sort_values('Total NFe',ascending=False,ignore_index=True,inplace=True)

valores=[]

soma=0

total=tabelas_df['Consolidado']['Total NFe'].sum()

for val in tabelas_df['Consolidado']['Total NFe'].tolist():
    
    soma+=val
    
    valores.append(soma)    
    
    pass

tabelas_df['Consolidado']['Acumulado']=valores

tabelas_df['Consolidado']['Total']=total

tabelas_df['Consolidado']['Perc %']=round(tabelas_df['Consolidado']['Acumulado']/tabelas_df['Consolidado']['Total'],4)*100

tabelas_df['Consolidado']['Classificação']=tabelas_df['Consolidado']['Perc %'].apply(Classificacao)

tabelas_df['Classificação ABC']=tabelas_df['Consolidado'].groupby(['Classificação'],as_index=False).agg({'Total NFe':'sum','SKU':'count'})

tabelas_df['Classificação ABC']

,Classificação,Total NFe,SKU
0,A,388208.40,13
1,B,73622.69,22
2,C,25124.04,23


In [39]:
perc_real=round(val_semana/meta_semanal,4)*100 if meta_semanal>0 else 0

perc_real

0

In [40]:
dif_semanal=val_semana-meta_semanal


dif_semanal

0.0

In [41]:
val_total=round(tabelas_df['SELLIN']['Total NFe'].loc[(tabelas_df['SELLIN']['Tipo']=='PADRAO')&(tabelas_df['SELLIN']['Data de Recebimento'].dt.year==datetime.now().year)&(tabelas_df['SELLIN']['Data de Recebimento'].dt.month==datetime.now().month)].sum(),2)

val_total

0.0

In [43]:
perc_total=round(val_total/val_meta,4)*100 if val_meta>0 else 0

perc_total

0

In [45]:
dif_mes=val_total-val_meta

dif_mes

0.0

In [46]:
tabelas_df['Lista']=tabelas_df['SELLIN'][['SKU','Produto','Fabricante','Total NFe']].loc[(tabelas_df['SELLIN']['Tipo']=='PADRAO')&(tabelas_df['SELLIN']['Data de Recebimento'].isin(list_datas))].groupby(['SKU','Produto','Fabricante'],as_index=False).sum()

tabelas_df['Consolidado']=tabelas_df['Consolidado'][['SKU','Classificação']]

tabelas_df['Lista']=tabelas_df['Lista'].merge(tabelas_df['Consolidado'],on='SKU',how='left')

tabelas_df['Lista'].loc[tabelas_df['Lista']['Classificação'].isnull(),'Classificação']='C'

tabelas_df['Lista'].sort_values('Total NFe',ascending=False,ignore_index=True,inplace=True)

tabelas_df['Lista']#.loc[tabelas_df['Lista']['Classificação']!='C']

,Produto,Fabricante,Total NFe,SKU,Classificação


In [47]:
tabelas_df['Classificação ABC']['Entradas']=tabelas_df['Classificação ABC']['Classificação'].apply(lambda info: tabelas_df['Lista']['Total NFe'].loc[tabelas_df['Lista']['Classificação']==info].sum())

tabelas_df['Classificação ABC']['Produtos']=tabelas_df['Classificação ABC']['Classificação'].apply(lambda info: len(tabelas_df['Lista']['SKU'].loc[tabelas_df['Lista']['Classificação']==info].unique().tolist()))

tabelas_df['Classificação ABC']=tabelas_df['Classificação ABC'][['Classificação','Entradas','Produtos']]

In [48]:
total=round(tabelas_df['Classificação ABC']['Entradas'].sum(),2)

tabelas_df['Classificação ABC']['Perc %']=round(tabelas_df['Classificação ABC']['Entradas']/total,4)*100

tabelas_df['Classificação ABC'].loc[tabelas_df['Classificação ABC']['Perc %'].isnull(),'Perc %']=0

tabelas_df['Classificação ABC']

C:\Users\eduardo\AppData\Local\Temp/ipykernel_6052/531074667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabelas_df['Classificação ABC']['Perc %']=round(tabelas_df['Classificação ABC']['Entradas']/total,4)*100
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Classificação,Entradas,Produtos,Perc %
0,A,0.0,0,0.0
1,B,0.0,0,0.0
2,C,0.0,0,0.0
